In [1]:
import tensorflow as tf

# Limit GPU memory usage
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=(6 * 1024))])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2025-05-13 01:58:24.987750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 01:58:24.997747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747067305.008861   65743 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747067305.012688   65743 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747067305.021199   65743 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1747067306.861185   65743 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6144 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
from tensorflow.keras import layers, models
import numpy as np

In [3]:
rating_dim = 3          # e.g., service, cleanliness, value
embedding_dim = 128
lstm_units = 256
vocab_size = 5000       # change depending on tokenizer
max_seq_len = 20        # maximum length of output text


In [4]:
def build_model(rating_dim, embedding_dim, lstm_units, vocab_size, max_seq_len):
    # Inputs
    rating_input = layers.Input(shape=(rating_dim,), name="ratings")
    text_input = layers.Input(shape=(max_seq_len,), name="text")

    # Process ratings
    rating_proj = layers.Dense(lstm_units, activation="relu")(rating_input)
    rating_proj = layers.RepeatVector(max_seq_len)(rating_proj)  # [batch, seq_len, lstm_units]

    # Process tokens
    text_embed = layers.Embedding(vocab_size, embedding_dim)(text_input)

    # Combine ratings and text
    lstm_input = layers.Concatenate()([text_embed, rating_proj])

    # LSTM Decoder
    lstm_output = layers.LSTM(lstm_units, return_sequences=True)(lstm_input)
    output = layers.TimeDistributed(layers.Dense(vocab_size, activation="softmax"))(lstm_output)

    model = models.Model(inputs=[rating_input, text_input], outputs=output)
    return model


In [5]:
model = build_model(rating_dim, embedding_dim, lstm_units, vocab_size, max_seq_len)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ratings             │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text (InputLayer)   │ (None, 20)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │      1,024 │ ratings[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 128)   │    640,000 │ text[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 20, 256)   │          0 │ dense[0][0]       │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 20, 384)   │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 20, 256)   │    656,384 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 20, 5000)  │  1,285,000 │ lstm[0][0]        │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,582,408 (9.85 MB)

 Trainable params: 2,582,408 (9.85 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
sample_data = [
    {"ratings": [5.0, 5.0, 5.0], "review": "Excellent service and very clean room."},
    {"ratings": [4.0, 3.5, 4.5], "review": "Good experience, but the room could be cleaner."},
    {"ratings": [2.0, 2.5, 2.0], "review": "Dirty room and poor service."},
    {"ratings": [3.0, 4.0, 3.5], "review": "Room was okay and fairly clean."},
    {"ratings": [1.0, 1.5, 1.0], "review": "Terrible experience. Not worth the money."},
    {"ratings": [4.5, 4.5, 4.0], "review": "Very clean and staff were friendly."},
]

In [ ]:
import json
import os

import pandas as pd
import numpy as np

def read_json_to_df(file_name):
    data = []
    with open(file_name) as data_file:
        for line in data_file:
            # Load each line of the JSON file as a dictionary
            data.append(json.loads(line))

    # Form a Pandas DataFrame from the dictionaries
    return pd.json_normalize(data)

# Load the training and test data
raw_train_df = read_json_to_df("hotel_reviews_train.json")
raw_test_df = read_json_to_df("hotel_reviews_test.json")

ratings_columns = [col for col in raw_train_df.columns if col.startswith("ratings.")]

# Select the title, text and overall rating columns to make a new dataframe
train_df = raw_train_df[["title", "text"] + ratings_columns]
test_df = raw_test_df[["title", "text"] + ratings_columns]

# Save the English reviews to a CSV file to save time filtering when running again (NumFOCUS, Inc. 2024)
if os.path.exists("english_hotel_reviews_train.csv"):
    train_df = pd.read_csv("english_hotel_reviews_train.csv")

if os.path.exists("english_hotel_reviews_test.csv"):
    test_df = pd.read_csv("english_hotel_reviews_test.csv")

train_df_2 = train_df.fillna(0)

inputs = train_df_2[ratings_columns]
# outputs = train_df_2['title'] + ' ' + train_df_2['text']
outputs = train_df_2['text']

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

# Ensure 'inputs' has no NaNs
inputs = train_df_2[ratings_columns].fillna(0).astype(np.float32).values

# Use review text as output
outputs = train_df_2['text'].astype(str).values  # Ensure string type

# Add special tokens
texts_with_tokens = ["<start> " + text + " <end>" for text in outputs]

# Tokenize text with special tokens
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts_with_tokens)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(texts_with_tokens)

# Define max length for padding
max_len = 21  # or use max_len = max(len(seq) for seq in sequences)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# Create input and target sequences (for teacher forcing in seq2seq models)
input_seq = padded_sequences[:, :-1]
target_seq = padded_sequences[:, 1:]


In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, RepeatVector, TimeDistributed

# Key dimensions
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
seq_length = input_seq.shape[1]
num_ratings = inputs.shape[1]

# Ratings input branch
ratings_input = Input(shape=(num_ratings,), name="ratings_input")
ratings_dense = Dense(64, activation='relu')(ratings_input)
ratings_repeated = RepeatVector(seq_length)(ratings_dense)

# Text input branch
text_input = Input(shape=(seq_length,), name="text_input")
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length)(text_input)

# Combine both inputs
merged = Concatenate()([embedding, ratings_repeated])

# Decoder LSTM
lstm_out = LSTM(256, return_sequences=True)(merged)
output = TimeDistributed(Dense(vocab_size, activation='softmax'))(lstm_out)

# Build and compile model
model = Model(inputs=[ratings_input, text_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


/home/dev/repos/COMP-SCI-7417-Applied-Natural-Language-Processing/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ratings_input       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_input          │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │        640 │ ratings_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 20, 100)   │  4,569,800 │ text_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 20, 64)    │          0 │ dense_2[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 20, 164)   │          0 │ embedding_1[0][0… │
│ (Concatenate)       │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 20, 256)   │    431,104 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 20, 45698) │ 11,744,386 │ lstm_1[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,745,930 (63.88 MB)

 Trainable params: 16,745,930 (63.88 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
target_seq = np.expand_dims(target_seq, -1)  # shape: (samples, timesteps, 1)

In [21]:
model.fit(
    [inputs, input_seq],  # ratings + input tokens
    target_seq,
    batch_size=32,
    epochs=10,
    validation_split=0.2
)


Epoch 1/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 40s 85ms/step - loss: 6.6521 - val_loss: 6.0070
Epoch 2/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 86ms/step - loss: 5.8988 - val_loss: 5.6881
Epoch 3/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 87ms/step - loss: 5.6310 - val_loss: 5.5231
Epoch 4/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 85ms/step - loss: 5.4253 - val_loss: 5.1867
Epoch 5/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 85ms/step - loss: 5.0562 - val_loss: 4.9067
Epoch 6/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 86ms/step - loss: 4.7652 - val_loss: 4.7282
Epoch 7/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 86ms/step - loss: 4.5974 - val_loss: 4.6163
Epoch 8/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 86ms/step - loss: 4.4470 - val_loss: 4.5416
Epoch 9/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 38s 85ms/step - loss: 4.3366 - val_loss: 4.4734
Epoch 10/10
453/453 ━━━━━━━━━━━━━━━━━━━━ 39s 87ms/step - loss: 4.2385 - val_loss: 4.4324


In [22]:
def generate_review(model, tokenizer, ratings_input, max_len=20, temperature=0.7, max_repeats=3):
    # Get start/end token IDs safely
    start_token = tokenizer.word_index.get('start')  
    end_token = tokenizer.word_index.get('end')     

    if start_token is None or end_token is None:
        raise ValueError("Tokenizer is missing 'start' or 'end' tokens. Make sure you added them during training.")

    input_seq = [start_token]
    generated_words = []  # To store the generated words
    generated_token_ids = set()  # To track generated token IDs and avoid repetition

    reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}  # Reverse index for decoding tokens

    # Debugging: Check the reverse word index
    print(f"Reverse word index size: {len(reverse_word_index)}")
    print(f"Reverse word index sample: {list(reverse_word_index.items())[:20]}")  # Print first 20 items

    repeat_count = 0  # Track repetition of words
    for _ in range(max_len):
        padded_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=max_len, padding='post')

        preds = model.predict([ratings_input, padded_seq], verbose=0)
        
        # Apply temperature to the predictions
        preds = preds[0][len(input_seq) - 1]  # Get prediction for the next word
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(preds / temperature)  # Apply temperature
        preds = preds / np.sum(preds)  # Normalize to sum to 1 (probabilities)

        # Limit predictions to valid token IDs (tokens in the word index)
        valid_tokens = list(tokenizer.word_index.values())  # Get list of all valid token IDs
        valid_preds = preds[valid_tokens]  # Get the prediction probabilities for valid tokens
        valid_preds /= np.sum(valid_preds)  # Normalize the valid tokens' probabilities

        # Sample a token from the valid predictions
        next_token_id = np.random.choice(valid_tokens, p=valid_preds)

        # Map token ID to word using reverse_word_index
        next_word = reverse_word_index.get(next_token_id, None)

        # Debugging line: Check predicted token and its word
        print(f"Predicted token ID: {next_token_id} -> Word: {next_word}")

        # If the predicted word is invalid or None, skip this iteration or stop early
        if next_word is None:  
            print("Prediction is None, stopping early...")
            break

        # Avoid repetition of the same token (in case of overly repetitive predictions)
        if next_token_id in generated_token_ids:
            repeat_count += 1
        else:
            repeat_count = 0
        
        if repeat_count > max_repeats:  # Stop if the model repeats the same word too much
            print("Model is repeating tokens too often. Stopping early...")
            break

        # Stop at end token
        if next_token_id == end_token:
            break

        generated_words.append(next_word)
        input_seq.append(next_token_id)
        generated_token_ids.add(next_token_id)

    # Convert list of words back to a string
    generated_review = ' '.join(generated_words).strip()

    print(f"Generated review: {generated_review}")
    return generated_review


In [26]:
test_rating = np.array([[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]])  # Example rating
print(seq_length)
generated_review = generate_review(model, tokenizer, test_rating, max_len=20)

20
Reverse word index size: 45697
Reverse word index sample: [(1, '<OOV>'), (2, 'the'), (3, 'and'), (4, 'a'), (5, 'to'), (6, 'was'), (7, 'i'), (8, 'in'), (9, 'we'), (10, 'of'), (11, 'is'), (12, 'for'), (13, 'hotel'), (14, 'it'), (15, 'room'), (16, 'at'), (17, 'were'), (18, 'but'), (19, 'with'), (20, 'on')]
Predicted token ID: 17619 -> Word: clarified
Predicted token ID: 13998 -> Word: kitschy
Predicted token ID: 27637 -> Word: samething
Predicted token ID: 42061 -> Word: midde
Predicted token ID: 26217 -> Word: 2499
Predicted token ID: 3683 -> Word: numbers
Predicted token ID: 38493 -> Word: soonwe
Predicted token ID: 41294 -> Word: sublease
Predicted token ID: 31778 -> Word: josephine
Predicted token ID: 23433 -> Word: joette
Predicted token ID: 2527 -> Word: 250
Predicted token ID: 27017 -> Word: irregular
Predicted token ID: 16146 -> Word: lynn
Predicted token ID: 26168 -> Word: blooks
Predicted token ID: 35366 -> Word: peered
Predicted token ID: 27879 -> Word: professionalone
Predi

In [11]:
# Text summarisation example
from transformers import TFBartForConditionalGeneration, BartTokenizer, TFTrainingArguments
import tensorflow as tf
import numpy as np

def format_input(ratings):
    return f"service: {ratings[0]} cleanliness: {ratings[1]} value: {ratings[2]}"

inputs = [format_input(d["ratings"]) for d in sample_data]
targets = [d["review"] for d in sample_data]

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Tokenize
input_encodings = tokenizer(inputs, padding="max_length", truncation=True, max_length=32, return_tensors="tf")
target_encodings = tokenizer(targets, padding="max_length", truncation=True, max_length=32, return_tensors="tf")

IGNORE_INDEX = -100
labels = tf.where(
    target_encodings.input_ids == tokenizer.pad_token_id,
    tf.constant(IGNORE_INDEX, dtype=tf.int32),
    target_encodings.input_ids,
)

# Prepare dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": input_encodings.input_ids,
        "attention_mask": input_encodings.attention_mask,
        "decoder_input_ids": target_encodings.input_ids,
        "decoder_attention_mask": target_encodings.attention_mask,
    },
    labels,
)).shuffle(10).batch(2)

# Compile model manually
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")

@tf.function
def train_step(batch_inputs, batch_labels):
    with tf.GradientTape() as tape:
        outputs = model(
            input_ids=batch_inputs["input_ids"],
            attention_mask=batch_inputs["attention_mask"],
            decoder_input_ids=batch_inputs["decoder_input_ids"],
            decoder_attention_mask=batch_inputs["decoder_attention_mask"],
            labels=batch_labels,
        )
        loss = tf.reduce_mean(outputs.loss)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Train
EPOCHS = 100
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    for step, (batch_inputs, batch_labels) in enumerate(dataset):
        loss = train_step(batch_inputs, batch_labels)
        print(f"  Step {step + 1}: loss = {loss.numpy():.4f}")

# Test generation
test_input = "service: 2.0 cleanliness: 1.0 value: 2.5"
test_encoding = tokenizer([test_input], return_tensors="tf", padding=True, truncation=True)
output_ids = model.generate(
    input_ids=test_encoding["input_ids"],
    attention_mask=test_encoding["attention_mask"],
    max_length=50
)
generated_review = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated review:", generated_review)

/home/dev/repos/COMP-SCI-7417-Applied-Natural-Language-Processing/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Epoch 1/100
  Step 1: loss = 9.3640
  Step 2: loss = 5.6177
  Step 3: loss = 6.0542
Epoch 2/100
  Step 1: loss = 3.3473


2025-05-13 01:58:54.355631: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Step 2: loss = 1.9492
  Step 3: loss = 2.3470
Epoch 3/100


2025-05-13 01:58:54.794185: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

  Step 1: loss = 0.0003
  Step 2: loss = 0.0002
  Step 3: loss = 0.0002
Epoch 34/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0003
Epoch 35/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0002
Epoch 36/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0002
Epoch 37/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0001
Epoch 38/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0001
Epoch 39/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0002
  Step 3: loss = 0.0002
Epoch 40/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0002
  Step 3: loss = 0.0001
Epoch 41/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0002
  Step 3: loss = 0.0002
Epoch 42/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 43/100
  Step 1: loss = 0.0002
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 44/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 

2025-05-13 01:59:21.895739: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 66/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 67/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 68/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 69/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 70/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 71/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 72/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 73/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 74/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 75/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Epoch 76/100
  Step 1: loss = 0.0001
  Step 2: loss = 0.0001
  Step 3: loss = 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  Step 2: loss = 0.0001
  Step 3: loss = 0.0001
Generated review: RoomRoomRoom Room Room Room room room room and and andandandand and andAndandandAndand andand andAnd and andD and and... and. and and ...
